# Using PreTrained InceptionV3, apply transfer learning to classify cat breeds #

### 1. Install ###

In [2]:
pip install --no-cache-dir pillow 

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


### 2. Import dependencies ###

In [3]:
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import SGD
import os.path

### 3. Defining Model ###

In [36]:
MODEL_FILE = 'Cats.keras'

In [37]:
# Create model from InceptionV3 and imagenet

def create_model(num_hidden, num_classes):
    
    base_model = InceptionV3(weights='imagenet', include_top=False)
    
    # get the output layer, does an avg pooling of output, 
    # and feed it to a final Dense Layer that we'll train
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(num_hidden, activation='relu')(x)
    predictions = Dense(num_classes, activation='softmax')(x)
    
    # Freeze base model layers
    for layer in base_model.layers:
        layer.trainable = False
    
    # Create a functional model
    model = Model(inputs=base_model.input, outputs=predictions)
    
    return model

### 4. Loading existing model ###

In [38]:
# Loads an existing model, but sets only the last 3 layers (which we added)
# to be trainable.

def load_existing(model_file):
    
    model = load_model(model_file)
    
    # set only the last 3 layers to be trainable
    num_layers = len(model.layers)
    for layer in model.layers[:num_layers-3]:
        layer.trainable = False
    for layer in model.layers[num_layers-3:]:
        layer.trainable = True
    
    return model

### 5. Training the Model ###   

In [39]:
# Trains a model, creates new model if not exists
def train(model_file, train_path, validation_path, num_hidden=200, num_classes=5, steps=32, num_epochs=20):
    if os.path.exists(model_file):
        print("\n--- Loading existing model: %s ---\n" % model_file)
        model = load_existing(model_file)
    else:
        print("\n--- Creating new model ---\n")
        model = create_model(num_hidden, num_classes)
        
    model.compile(loss='categorical_crossentropy',optimizer='rmsprop', metrics=['accuracy'])
    
    # create a checkpoint to save model after each epoch
    checkpoint = ModelCheckpoint(model_file)
    
    # creating a image generator for more training data
    train_datagen = ImageDataGenerator(rescale=1./255, shear_range=0.2, zoom_range=0.2, horizontal_flip=True, rotation_range=30)
    
    # now for test data
    test_datagen = ImageDataGenerator(rescale=1./255, shear_range=0.2, zoom_range=0.2, horizontal_flip=True, rotation_range=30)
    
    # Now we tell the generator where to get the images from.
    # We also scale the images to 249x249 pixels.
    train_generator = train_datagen.flow_from_directory(
        train_path,
        target_size=(249, 249),
        batch_size=5,
        class_mode="categorical"
    )
    
    # We do the same for the validation set.
    validation_generator = test_datagen.flow_from_directory(
        validation_path,
        target_size=(249, 249),
        batch_size=5,
        class_mode='categorical'
    )
    
    # Note that pictures must be sorted into different directories
    # the directories each must be named after the corresponding category
    
    # Finally we train the neural network
    model.fit(
        train_generator,
        steps_per_epoch = steps,
        epochs = num_epochs,
        callbacks = [checkpoint],
        validation_data = validation_generator,
        validation_steps = 50
    )
    print("\n --- Top Layers Trained, Begin Fine Tuning ---\n")

    # After training top layers, unfreeze some of the topmost base layers
    # and train together with the top layers
    
    for layer in model.layers[:249]:
        layer.trainable = False
    for layer in model.layers[249:]:
        layer.trainable = True
        
    model.compile(loss='categorical_crossentropy',optimizer=SGD(learning_rate=0.001, momentum=0.9),metrics=['accuracy'])
    
    model.fit(train_generator, steps_per_epoch = steps, epochs = num_epochs, callbacks = [checkpoint], validation_data = validation_generator, validation_steps = 50)
    

### 6. train.py with all above ###

In [40]:
def main():
    train(MODEL_FILE, train_path="photos",validation_path="photos",steps=120, num_epochs=5)

if __name__ == '__main__':
    main()


--- Creating new model ---

Found 323 images belonging to 5 classes.
Found 323 images belonging to 5 classes.
Epoch 1/5
120/120 ━━━━━━━━━━━━━━━━━━━━ 29s 189ms/step - accuracy: 0.6158 - loss: 1.7607 - val_accuracy: 0.5920 - val_loss: 1.2393
Epoch 2/5
120/120 ━━━━━━━━━━━━━━━━━━━━ 14s 110ms/step - accuracy: 0.7863 - loss: 0.6302 - val_accuracy: 0.8904 - val_loss: 0.3005
Epoch 3/5
120/120 ━━━━━━━━━━━━━━━━━━━━ 18s 148ms/step - accuracy: 0.8708 - loss: 0.3122 - val_accuracy: 0.9080 - val_loss: 0.2637
Epoch 4/5
120/120 ━━━━━━━━━━━━━━━━━━━━ 15s 118ms/step - accuracy: 0.8633 - loss: 0.3755 - val_accuracy: 0.9452 - val_loss: 0.1932
Epoch 5/5
120/120 ━━━━━━━━━━━━━━━━━━━━ 20s 159ms/step - accuracy: 0.8699 - loss: 0.3800 - val_accuracy: 0.9280 - val_loss: 0.1379

 --- Top Layers Trained, Begin Fine Tuning ---

Epoch 1/5
120/120 ━━━━━━━━━━━━━━━━━━━━ 34s 217ms/step - accuracy: 0.7912 - loss: 0.5205 - val_accuracy: 0.8320 - val_loss: 0.5475
Epoch 2/5
120/120 ━━━━━━━━━━━━━━━━━━━━ 19s 157ms/step - accu

### 7. Predicting with model ###

In [44]:
MODEL_NAME = 'Cats.keras'

In [45]:
from tensorflow.keras.models import load_model
import tensorflow as tf
from tensorflow.python.keras.backend import set_session
import numpy as np
from PIL import Image
from os import listdir
from os.path import join

# Our samples directory
SAMPLE_PATH = './samples'

dict = {0: 'Pallas', 1: 'Persian', 2: 'Ragdolls', 3: 'Singapura', 4: 'Sphynx'}

# Takes in a loaded model, an image in numpy matrix format,
# And a label dictionary
def classify(model, image):
    result = model.predict(image)
    themax = np.argmax(result)
    return (dict[themax], result[0][themax], themax)

# Load image
def load_image(image_fname):
    img = Image.open(image_fname)
    img = img.resize((249, 249))
    imgarray = np.array(img) / 255.0
    final = np.expand_dims(imgarray, axis=0)
    return final


In [46]:
# Test main
def main():
    print("\n ---Loading model from %s ---\n"% MODEL_NAME)
    model = load_model(MODEL_NAME)
    print("Done")

    print("\n ---Now classifying files in %s ---\n"% SAMPLE_PATH)
    
    sample_files = listdir(SAMPLE_PATH)

    for filename in sample_files:
        filename = join(SAMPLE_PATH, filename)
        img = load_image(filename)
        label, prob, _ = classify(model, img)
        
        print("We think with certainty %3.2f that image %s is %s." % (prob, filename, label))

if __name__ == "__main__":
    main()



 ---Loading model from Cats.keras ---

Done

 ---Now classifying files in ./samples ---

1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
We think with certainty 1.00 that image ./samples\Pallas cats_115.jpg is Pallas.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
We think with certainty 1.00 that image ./samples\Pallas cats_144.jpg is Pallas.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step
We think with certainty 0.99 that image ./samples\Pallas cats_76.jpg is Pallas.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
We think with certainty 1.00 that image ./samples\Persian cats_225.jpg is Ragdolls.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
We think with certainty 0.98 that image ./samples\Persian cats_44.jpg is Persian.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step
We think with certainty 1.00 that image ./samples\Persian cats_88.jpg is Persian.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step
We think with certainty 1.00 that image ./samples\Ragdolls_215.jpg is Ragdolls.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
We think with certainty 1.00 that imag